# Mapping components from the iBuildGreen database to LCAbyg constructions and products

This notebook shows how components in the iBuildGreen database (i.e. types of roofs, external walls, etc) are linked with constructions and products from LCAbyg. Using LCAbyg is useful because it provides a user-friendly interface to create and modify components, as well as an already existing database of constructions and products. Furthermore, ensuring compatibility with LCAbyg is a safe choice: it will allow us to link the two programs more closely in the future, to use LCAbyg to perform environmental calculations in iBuildGreen, to also import data on "stages" from LCAbyg if needed, etc.

In LCAbyg, buildings are defined as sets of "elements". Each element is made of "constructions", and each construction is made of "products". Each product is then described as a set of LCA "stages". In this notebook, we will only be importing "construction" and "products" for now. We will link each iBuildGreen macrocomponent with one or several constructions, and each construction with one or several products.

# Setup

In [3]:
import json
import psycopg
params='dbname=macrocomponents user=postgres password=19Ni93co44PG!' # Write the parameters to connect to your database here

# Constructions

This section shows how to import "constructions" from LCAbyg into the database. First, we need to read the json file containing data on these constructions.

In [2]:
# Write here the location of the LCAbyg "constructions" json file on your computer
with open('C:/Users/KJ35FA/OneDrive - Aalborg Universitet/Documents/Available building data/LCAbyg/constructions.json') as json_data:
    const = json.load(json_data) # Load LCAbyg constructions

In [ ]:
for c in const: # For information - display the names of all constructions
    for key in c.keys():        
        if key=='Node':
            name=c[key]['Construction']['name']['English']
            print(name)        

In iBuildGreen database, we leave open the possibility that each macrocomponent (e.g. external wall type) could be linked with one or several LCAbyg constructions. So we insert LCAbyg constructions into a separate table, called "subcomponents". Later, we will then link macrocomponents with the subcomponents table.

First, we define a function to insert items in the subcomponents table.

In [8]:
def insert_subcomponent(list_of_rows):
    sql = "INSERT INTO subcomponents(lcabyg_id, name, unit, layer, comment) VALUES(%s, %s, %s, %s, %s) ON CONFLICT ON CONSTRAINT subcomponents_pkey DO UPDATE SET (lcabyg_id, name, unit, layer, comment) = (EXCLUDED.lcabyg_id, EXCLUDED.name, EXCLUDED.unit, EXCLUDED.layer, EXCLUDED.comment);"
    connector = None
    bbrid = None
    try:
        # connect to the PostgreSQL database
        connector = psycopg.connect(params)
        # create a new cursor
        cur = connector.cursor()
        # execute the INSERT statement
        cur.executemany(sql, list_of_rows)
        # commit the changes to the database
        connector.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg.DatabaseError) as error:
        print(error)
    finally:
        if connector is not None:
            connector.close()

Then, we retrieve our subcomponents from the imported json object, as a list:

In [6]:
list_of_const=[]

for c in const:
    for key in c.keys():        
        if key=='Node':
            ID=c[key]['Construction']['id']
            name=c[key]['Construction']['name']['English']
            unit=c[key]['Construction']['unit']
            layer=c[key]['Construction']['layer']
            comment=c[key]['Construction']['comment']['Danish']
            list_of_rows.append((ID,name,unit,layer,comment))

Finally, we insert this list of rows into the database using the function we just defined:

In [ ]:
insert_subcomponent(list_of_const)

# Products

As above, we start by importing the json file containing data on LCAbyg products:

In [11]:
with open('C:/Users/KJ35FA/OneDrive - Aalborg Universitet/Documents/Available building data/LCAbyg/products.json') as json_data:
    prod = json.load(json_data) 

In [ ]:
for c in prod: # For information - display the names of all products
    for k,p in c.items():        
         print(k,p)

Then we define a function to insert products into the database:

In [9]:
def insert_product(list_of_prods):
    sql = "INSERT INTO products(lcabyg_id, name, comment) VALUES(%s, %s, %s) ON CONFLICT ON CONSTRAINT products_pkey DO UPDATE SET (lcabyg_id, name, comment) = (EXCLUDED.lcabyg_id, EXCLUDED.name, EXCLUDED.comment);"
    connector = None
    bbrid = None
    try:
        # connect to the PostgreSQL database
        connector = psycopg.connect(params)
        # create a new cursor
        cur = connector.cursor()
        # execute the INSERT statement
        cur.executemany(sql, list_of_prods)
        # commit the changes to the database
        connector.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg.DatabaseError) as error:
        print(error)
    finally:
        if connector is not None:
            connector.close()

Finally we retrieve information about products from the json object into a list, and we insert this list using the function above.

In [12]:
list_of_prods=[]
for p in prod:
    for key in p.keys():        
        if key=='Node':
            ID=p[key]['Product']['id']
            name=p[key]['Product']['name']['English']
            comment=p[key]['Product']['comment']
            list_of_prods.append((ID,name,comment))

In [ ]:
insert_product(list_of_prods)

# Mapping table

We have just imported constructions and products into the database, but for now we haven't imported any information on how the two are linked together: we don't know which products are in which constructions. We will now solve this by filling a mapping table "subcomponents_to_products". Each row of the mapping table will indicate the id of a construction, the id of a product, and the amount of this particular product found in this particular construction.

We start by clearing the mapping table if it wasn't already empty:

In [ ]:
try:
    connector = psycopg.connect(params)
    cur = connector.cursor()
    cur.execute("DELETE FROM subcomponents_to_products")
    connector.commit()
    cur.close()
except (Exception, psycopg.DatabaseError) as error:
    print(error)
finally:
    if connector is not None:
        connector.close()

Links between constructions and products are recorded in the LCAbyg "constructions" json file. They are objects of type "Edge". We can start by retrieving these edges from the "constructions" json file:

In [ ]:
list_of_edges=[]
for c in const:
    for key in c.keys():        
        if key=='Edge':
            ID=c[key][0]['ConstructionToProduct']['id']
            amount=c[key][0]['ConstructionToProduct']['amount']
            unit=c[key][0]['ConstructionToProduct']['unit']
            lifespan=c[key][0]['ConstructionToProduct']['lifespan']
            const_id=c[key][1]
            prod_id=c[key][2]
            list_of_edges.append((ID,const_id,prod_id,amount,unit,lifespan))

Then, as above, we define a function to insert elements into the mapping table:

In [15]:
def map_component_product(list_of_edges):
    sql = "INSERT INTO subcomponents_to_products(id, subcomponent_id, product_id, amount, unit, lifespan) VALUES(%s, %s, %s, %s, %s, %s) ON CONFLICT ON CONSTRAINT subcomponents_to_products_pkey DO UPDATE SET (id, subcomponent_id, product_id, amount, unit, lifespan) = (EXCLUDED.id, EXCLUDED.subcomponent_id, EXCLUDED.product_id, EXCLUDED.amount, EXCLUDED.unit, EXCLUDED.lifespan);"
    try:
        # connect to the PostgreSQL database
        connector = psycopg.connect(params)
        
        # fetch construction ids
        cur_const = connector.cursor()
        cur_const.execute('SELECT lcabyg_id FROM subcomponents')
        const_ids=[]
        for i in cur_const.fetchall():
            const_ids.append(i[0])
            
        # fetch product ids       
        cur_prod = connector.cursor()
        cur_prod.execute('SELECT lcabyg_id FROM products')
        prod_ids=[]
        for i in cur_prod.fetchall():
            prod_ids.append(i[0])
        
        # checking for foreign key constraints
        # The LCAbyg edges may include references to products or constructions that are not recorded in the respective files,
        # for instance because a product has been deleted but the corresponding edge hasn't. We need to deal with these cases separately,
        # otherwise the insertion query will raise foreign key constraints (we are trying to insert a row referring to products that do not exist)
        
        valid_edges=[]
        missing_subcomponents=[]
        missing_products=[]
        for edge in list_of_edges:
            if edge[1] in const_ids and edge[2] in prod_ids:
                valid_edges.append(edge)
            if edge[1] not in const_ids:
                missing_subcomponents.append((edge[1],))
            if edge[2] not in prod_ids:
                missing_products.append((edge[2],))
                      
        # insert missing subcomponents
        cur = connector.cursor()
        sql_const = "INSERT INTO subcomponents(lcabyg_id, name) VALUES(%s, 'missing subcomponent') ON CONFLICT ON CONSTRAINT subcomponents_pkey DO NOTHING"
        cur.executemany(sql_const, missing_subcomponents)
        
        # insert missing products
        sql_prod = "INSERT INTO products(lcabyg_id, name) VALUES(%s, 'missing product') ON CONFLICT ON CONSTRAINT products_pkey DO NOTHING"
        cur.executemany(sql_prod, missing_products)
        
        # insert edges in mapping table
        cur.executemany(sql, list_of_edges)
        
        # commit the changes to the database
        connector.commit()
        # close communication with the database
        cur.close()
        cur_const.close()
        cur_prod.close()
    except (Exception, psycopg.DatabaseError) as error:
        print(error)
    finally:
        if connector is not None:
            connector.close()

Finally we insert the list of edges we just retrieved:

In [ ]:
map_component_product(list_of_edges)

# Extra components

LCAbyg also includes other data files linked with specific projects. These include additional constructions and components, that we might as well import.

In [21]:
with open('C:/Users/KJ35FA/OneDrive - Aalborg Universitet/Documents/Available building data/LCAbyg/constructions_new_br.json') as json_data:
    const_new = json.load(json_data) 

In [22]:
list_comp_new=[]
for c in const_new:
    for key in c.keys():        
        if key=='Node':
            ID=c[key]['Construction']['id']
            name=c[key]['Construction']['name']['English']
            unit=c[key]['Construction']['unit']
            layer=c[key]['Construction']['layer']
            comment=c[key]['Construction']['comment']['Danish']
            list_comp_new.append((ID,name,unit,layer,comment))

In [ ]:
insert_subcomponent(list_comp_new)

In [24]:
list_edges_new=[]
for c in const_new:
    for key in c.keys():        
        if key=='Edge':
            if 'ConstructionToProduct' in c[key][0].keys():
                ID=c[key][0]['ConstructionToProduct']['id']
                amount=c[key][0]['ConstructionToProduct']['amount']
                unit=c[key][0]['ConstructionToProduct']['unit']
                lifespan=c[key][0]['ConstructionToProduct']['lifespan']
                const_id=c[key][1]
                prod_id=c[key][2]
                list_edges_new.append((ID,const_id,prod_id,amount,unit,lifespan))

In [ ]:
map_component_product(list_edges_new)

In [27]:
with open('C:/Users/KJ35FA/OneDrive - Aalborg Universitet/Documents/Available building data/LCAbyg/enf_constructions.json') as json_data:
    const_enf = json.load(json_data) 
    
list_comp_enf=[]
for c in const_enf:
    for key in c.keys():        
        if key=='Node':
            if 'Construction' in c[key].keys():
                ID=c[key]['Construction']['id']
                name=c[key]['Construction']['name']['English']
                unit=c[key]['Construction']['unit']
                comment=c[key]['Construction']['comment']['Danish']
                if 'layer' in c[key]['Construction'].keys():
                    layer=c[key]['Construction']['layer']
                else:
                    layer=None
                list_comp_enf.append((ID,name,unit,layer,comment))
    
list_edges_enf=[]
for c in const_enf:
    for key in c.keys():        
        if key=='Edge':
            if 'ConstructionToProduct' in c[key][0].keys():
                ID=c[key][0]['ConstructionToProduct']['id']
                amount=c[key][0]['ConstructionToProduct']['amount']
                unit=c[key][0]['ConstructionToProduct']['unit']
                lifespan=c[key][0]['ConstructionToProduct']['lifespan']
                const_id=c[key][1]
                prod_id=c[key][2]
                list_edges_enf.append((ID,const_id,prod_id,amount,unit,lifespan))


In [ ]:
insert_subcomponent(list_comp_enf)
map_component_product(list_edges_enf)

In [31]:
with open('C:/Users/KJ35FA/OneDrive - Aalborg Universitet/Documents/Available building data/LCAbyg/epd_products.json') as json_data:
    epd_products = json.load(json_data) 

list_prods_epd=[]
for p in epd_products:
    for key in p.keys():        
        if key=='Node':
            if 'Product' in p[key].keys():
                ID=p[key]['Product']['id']
                name=p[key]['Product']['name']['English']
                comment=p[key]['Product']['comment']
                list_prods_epd.append((ID,name,comment)) 

In [ ]:
insert_product(list_prods_epd)

In [34]:
with open('C:/Users/KJ35FA/OneDrive - Aalborg Universitet/Documents/Available building data/LCAbyg/reno_ex_constructions.json') as json_data:
    reno_ex_constructions = json.load(json_data) 
    
list_comp_ex=[]
for c in reno_ex_constructions:
    for key in c.keys():        
        if key=='Node':
            if 'Construction' in c[key].keys():
                ID=c[key]['Construction']['id']
                name=c[key]['Construction']['name']['English']
                unit=c[key]['Construction']['unit']
                comment=c[key]['Construction']['comment']['Danish']
                if 'layer' in c[key]['Construction'].keys():
                    layer=c[key]['Construction']['layer']
                else:
                    layer=None
                list_comp_ex.append((ID,name,unit,layer,comment))
    
list_edges_ex=[]
for c in reno_ex_constructions:
    for key in c.keys():        
        if key=='Edge':
            if 'ConstructionToProduct' in c[key][0].keys():
                ID=c[key][0]['ConstructionToProduct']['id']
                amount=c[key][0]['ConstructionToProduct']['amount']
                unit=c[key][0]['ConstructionToProduct']['unit']
                lifespan=c[key][0]['ConstructionToProduct']['lifespan']
                const_id=c[key][1]
                prod_id=c[key][2]
                list_edges_ex.append((ID,const_id,prod_id,amount,unit,lifespan))

In [ ]:
insert_subcomponent(list_comp_ex)

map_component_product(list_edges_ex)

In [37]:
with open('C:/Users/KJ35FA/OneDrive - Aalborg Universitet/Documents/Available building data/LCAbyg/reno_reno_constructions.json') as json_data:
    reno = json.load(json_data) 

list_comp_reno=[]
for c in reno:
    for key in c.keys():        
        if key=='Node':
            if 'Construction' in c[key].keys():
                ID=c[key]['Construction']['id']
                name=c[key]['Construction']['name']['English']
                unit=c[key]['Construction']['unit']
                comment=c[key]['Construction']['comment']['Danish']
                if 'layer' in c[key]['Construction'].keys():
                    layer=c[key]['Construction']['layer']
                else:
                    layer=None
                list_comp_reno.append((ID,name,unit,layer,comment))
    
list_edges_reno=[]
for c in reno:
    for key in c.keys():        
        if key=='Edge':
            if 'ConstructionToProduct' in c[key][0].keys():
                ID=c[key][0]['ConstructionToProduct']['id']
                amount=c[key][0]['ConstructionToProduct']['amount']
                unit=c[key][0]['ConstructionToProduct']['unit']
                lifespan=c[key][0]['ConstructionToProduct']['lifespan']
                const_id=c[key][1]
                prod_id=c[key][2]
                list_edges_reno.append((ID,const_id,prod_id,amount,unit,lifespan))

In [ ]:
insert_subcomponent(list_comp_reno)

map_component_product(list_edges_reno)

# Handmade components

Because LCAbyg did not include appropriate constructions and products to describe all iBuildGreen macrocomponents, additional constructions and products were added manually. To preserve compatibility with LCAbyg, the constructions and products were created in LCAbyg, exported into a json file, and then imported into the database as previously:

In [ ]:
with open('C:/Users/KJ35FA/Documents/ibuildgreen_data.json') as json_data:
    byhand = json.load(json_data) 

list_prods_byhand=[]
for p in byhand:
    for key in p.keys():        
        if key=='Node':
            if 'Product' in p[key].keys():
                ID=p[key]['Product']['id']
                name=p[key]['Product']['name']['English']
                comment=p[key]['Product']['comment']['English']
                list_prods_byhand.append((ID,name,comment)) 
print(list_prods_byhand)

    
list_comp_byhand=[]
for c in byhand:
    for key in c.keys():        
        if key=='Node':
            if 'Construction' in c[key].keys():
                ID=c[key]['Construction']['id']
                name=c[key]['Construction']['name']['English']
                unit=c[key]['Construction']['unit']
                comment=c[key]['Construction']['comment']['Danish']
                if 'layer' in c[key]['Construction'].keys():
                    layer=c[key]['Construction']['layer']
                else:
                    layer=None
                list_comp_byhand.append((ID,name,unit,layer,comment))
print(list_comp_byhand)
    
list_edges_byhand=[]
for c in byhand:
    for key in c.keys():        
        if key=='Edge':
            if 'ConstructionToProduct' in c[key][0].keys():
                ID=c[key][0]['ConstructionToProduct']['id']
                amount=c[key][0]['ConstructionToProduct']['amount']
                unit=c[key][0]['ConstructionToProduct']['unit']
                lifespan=c[key][0]['ConstructionToProduct']['lifespan']
                const_id=c[key][1]
                prod_id=c[key][2]
                list_edges_byhand.append((ID,const_id,prod_id,amount,unit,lifespan))
print(list_edges_byhand)

In [ ]:
insert_product(list_prods_byhand)
insert_subcomponent(list_comp_byhand)

In [ ]:
map_component_product(list_edges_byhand)

# Mapping iBuildGreen macrocomponents to subcomponents/constructions

The link between iBuildGreen macrocomponents and subcomponents in the database was defined manually.